In [2]:
from datetime import date
%run ./pipeline_GCP.ipynb

def test_transformation_df():
    
    df = pd.DataFrame({
        "col1": [1, None, None, 1, 1],
        "col2": [1, None, None, 1, 2]
    })
    
    transformed = transformation_df(df)
    print("Résultat transformé :")
    print(transformed)
    

    assert "date_chargement" in transformed.columns
    assert transformed["date_chargement"].iloc[0] == date.today()
    assert len(transformed) == 2  
    assert pd.isna(transformed[['col1', 'col2']]).sum().sum() == 0  
    print("✅ Test FINAL réussi")

def test_sql_server_integration():
    """Teste que les données sont bien stockées dans SQL Server"""

    engine = create_sql_engine(
        server='DESKTOP-Q8EQVSL\\MSSQLSERVER2022',
        database='Fact_Commandes',
        username='aze',
        password='aze',
        driver='ODBC Driver 17 for SQL Server'
    )

    tables_to_check = [
        "dim_produit",
        "dim_entrepot",
        "stg_commandes",
        "stg_livrasion",
        "stg_mouvements",
        "FACT_Mouvement",
        "FACT_Livraison",
        "FACT_commandes"
    ]
    
    
    for table in tables_to_check:
      try:  
        df = pd.read_sql(f"SELECT TOP 1 * FROM {table}", engine)
        if  df.empty :
            logger.info(f"la  Table {table} est vide")
        else :
            logger.info(f"✅ Table {table} contient des données ")
      except Exception as e:
            logger.error(f"❌ Erreur sur la table {table} : {str(e)}")
            raise

def test_bigquery_integration():
    """Teste que les données sont bien stockées dans BigQuery"""

    credential_path = r"C:\Users\user\Pipeline_09_05_2025\credential_path\bigquery_credentials.json"
    credentials = service_account.Credentials.from_service_account_file(
    credential_path,
    scopes=["https://www.googleapis.com/auth/cloud-platform"],)
    client = bigquery.Client(credentials=credentials, project="pipeline-458019")
    
    

    tables_to_check = {
        "FACT_Mouvement": "vente",
        "FACT_Livraison": "vente",
        "FACT_commandes": "vente" ,
         "dim_produit" :"vente" ,
         "dim_entrepot" :"vente"
    }
    
    # 3. Vérification de chaque table
    for table, dataset in tables_to_check.items():
        try:
            query = f"SELECT * FROM `pipeline-458019.{dataset}.{table}` LIMIT 1"
            df = client.query(query).to_dataframe()
            assert not df.empty, f"La table {table} est vide"
            logger.info(f"✅ Table BigQuery {dataset}.{table} contient des données")
        except Exception as e:
            logger.error(f"❌ Erreur sur la table {dataset}.{table} : {str(e)}")
            raise

    logger.info("Test BigQuery réussi - Toutes les tables contiennent des données")

def main():
    test_transformation_df()
    test_sql_server_integration()
    test_bigquery_integration()

if __name__ == "__main__" :
    main()

2025-05-10 22:45:15,865 - __main__ - INFO - 🏁 Début du pipeline
2025-05-10 22:45:15,971 - __main__ - INFO - 🏁 Début chargement les fichiers
2025-05-10 22:45:15,990 - __main__ - INFO - 🏁 Début transformation les données
2025-05-10 22:45:19,795 - __main__ - INFO -  ---Chargement de dim_produit dans SQL Server---
2025-05-10 22:45:22,039 - __main__ - INFO -  ---Chargement de dim_entrepot dans SQL Server---
2025-05-10 22:45:23,049 - __main__ - INFO -  ---Chargement de stg_commandes dans SQL Server---
2025-05-10 22:45:24,081 - __main__ - INFO -  ---Chargement de stg_livrasion dans SQL Server---
2025-05-10 22:45:25,075 - __main__ - INFO -  ---Chargement de stg_mouvements dans SQL Server---
2025-05-10 22:45:25,982 - __main__ - INFO -  ---Chargement de FACT_Mouvement dans SQL Server---
2025-05-10 22:45:26,962 - __main__ - INFO -  ---Chargement de FACT_Livraison dans SQL Server---
2025-05-10 22:45:28,558 - __main__ - INFO -  ---Chargement de FACT_commandes dans SQL Server---


📤 Uploading FACT_Mouvement to BigQuery → pipeline-458019.vente.FACT_Mouvement ...
✅ Table FACT_Mouvement uploaded successfully.
📤 Uploading FACT_Livraison to BigQuery → pipeline-458019.vente.FACT_Livraison ...
✅ Table FACT_Livraison uploaded successfully.
📤 Uploading FACT_commandes to BigQuery → pipeline-458019.vente.FACT_commandes ...
✅ Table FACT_commandes uploaded successfully.
📤 Uploading dim_produit to BigQuery → pipeline-458019.vente.dim_produit ...
✅ Table dim_produit uploaded successfully.
📤 Uploading dim_entrepot to BigQuery → pipeline-458019.vente.dim_entrepot ...
✅ Table dim_entrepot uploaded successfully.
Résultat transformé :
   col1  col2 date_chargement
0   1.0   1.0      2025-05-10
4   1.0   2.0      2025-05-10
✅ Test FINAL réussi


2025-05-10 22:45:48,588 - __main__ - INFO - ✅ Table dim_produit contient des données 
2025-05-10 22:45:48,703 - __main__ - INFO - ✅ Table dim_entrepot contient des données 
2025-05-10 22:45:48,742 - __main__ - INFO - ✅ Table stg_commandes contient des données 
2025-05-10 22:45:48,776 - __main__ - INFO - ✅ Table stg_livrasion contient des données 
2025-05-10 22:45:48,900 - __main__ - INFO - ✅ Table stg_mouvements contient des données 
2025-05-10 22:45:49,887 - __main__ - INFO - ✅ Table FACT_Mouvement contient des données 
2025-05-10 22:45:49,918 - __main__ - INFO - ✅ Table FACT_Livraison contient des données 
2025-05-10 22:45:49,960 - __main__ - INFO - ✅ Table FACT_commandes contient des données 
2025-05-10 22:46:16,683 - __main__ - INFO - ✅ Table BigQuery vente.FACT_Mouvement contient des données
2025-05-10 22:46:19,072 - __main__ - INFO - ✅ Table BigQuery vente.FACT_Livraison contient des données
2025-05-10 22:46:21,234 - __main__ - INFO - ✅ Table BigQuery vente.FACT_commandes contien